# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!conda install -c conda-forge gmaps -y

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Owner\anaconda3\envs\PythonData

  added / updated specs:
    - gmaps


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       h5b45459_0         188 KB  conda-forge
    certifi-2022.6.15          |   py37h03978a9_0         156 KB  conda-forge
    openssl-1.1.1q             |       h8ffe710_0         5.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.1 MB

The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2022.4.26-h~ --> conda-forge::ca-certificates-2022.6.15-h5b45459_0
  certifi            anaconda::certifi-2021.10.8-py37haa95~ --> conda-forge::certifi-2022.6.15-py37h03978a9_0
  openssl               anaconda::openssl-1.1.1n-h2bbff1b_0 



==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base conda




In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
import json

In [20]:
! jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [22]:
! pip install gmaps

In [23]:
! jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacation_data =pd.read_csv("C:\\Users\\Owner\Documents\\Python-API-Challenge\\cities.csv")
vacation_df = pd.DataFrame(vacation_data)
vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Name,Date
0,0,sitka,57.0531,-135.3300,57.09,97,100,6.71,US,1660532104
1,1,pipri,19.8000,75.5333,73.98,88,75,6.91,IN,1660532104
2,2,cayenne,4.9333,-52.3333,81.09,1,100,1.14,GF,1660532105
3,3,esperance,-33.8667,121.9000,61.57,57,0,14.00,AU,1660532106
4,4,hobart,-42.8794,147.3294,53.22,95,75,12.66,AU,1660531832


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key and configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = vacation_df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
Humidity = vacation_df["Humidity"].astype(float)

In [88]:
# Customize the size of the figure
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)

In [7]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
hotel_df = vacation_df.loc[vacation_df['Max Temp'] > 72].loc[vacation_df['Max Temp']< 78].loc[vacation_df['Wind Speed'] < 10].loc[vacation_df['Cloudiness']==0]

In [9]:
len(hotel_df)


7

In [10]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Name,Date
14,14,burriana,39.8890,-0.0850,77.11,38,0,6.82,ES,1660532111
28,28,fernie,49.5000,-115.0687,75.40,42,0,3.80,CA,1660532117
322,322,rudsar,37.1376,50.2880,76.98,71,0,3.40,IR,1660532264
445,445,auburndale,28.0653,-81.7887,75.88,88,0,3.44,US,1660532055
446,446,porto walter,-8.2686,-72.7439,73.22,70,0,1.28,BR,1660532325
542,542,natal,-5.7950,-35.2094,73.62,83,0,6.91,BR,1660532343
554,554,evansville,37.9748,-87.5558,75.20,86,0,4.61,US,1660531739


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# set up additional columns to hold information
hotel_df['Hotel Name'] = ""

In [12]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotels = hotel_name.json()
    print(json.dumps(hotels, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
  

{
    "html_attributions": [],
    "next_page_token": "AeJbb3cdxmbWfBwubIIzljL92zALsWt7b5ABcffODUo5jXPfa7GCv6yVVYcPsiVDHVF0fr7_5QfYNW3CwNh70tkZdaMqSMUYIz6XTcT8Ca_xgHdHwpRkW7GUA7_Esmv0reETBwKc6P1E3P1fxbp9JMvtXwrUVO_p3ywE5CTow_1R95fhEfxtlDGpNYJmJsow0klvjNZxx7PclCUaIGZPldFFhrRwGYV2MvCMwrthcD4aTmM09g2RS0OhWMAM9b2C57aIL8CD8r_0qUXbZn9HT-Y6qD6KH60w1ro9RCmLuOnOUYlhjgRveDR5q1yMKlqff5pXrJjurWWUszQpP-UWIOcgyvbuzh7g3VUJd_4Pxbgo_Awt9ZJU_wl4wxhzejJt3CIe1XH2FFdN1uSUPILEnqpwKqxjqxgGcmWGw3CbunVEsqXcplkk_Or7tw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 39.5653877,
                    "lng": -0.2826758
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.56680012989272,
                        "lng": -0.2812828701072778
                    },
                    "southwest": {
                        "lat": 39.56410047010728,
 

{
    "html_attributions": [],
    "next_page_token": "AeJbb3cq0bLTRSEsz64ws-tXzYMc161-trKxXckoeObbB_dYF6Vg6m_jvm9hWxOJys1RXA85PGL7-IQF2LRXB8ifMKPq9xwyjy2nfxPT82CBmwcLO0ewsf3XYjm411qRHW2LG9S4Kk9nj-x9f_-mAJO09NT4LgafzEX5MDd_Vgo2pb5FVfbTzutXPcT-s9J3fB9VCwDPVtrKTcizgFsQm6EJU-dNfLkMDtdwCfaBiOfNoaaEHch-3T6YlRdIkTbSY5mSLHd0nFLI3iXLXiCx-T5sMDR-Dcm3C9GCzJP-Zl9akSVHEErQxXyOzAuYNKRva6VY60vnvforDrUzJLqDmZ4CnSoRu1G0eB4dFbe8yjkAZJyky8jB1k4pNYmdjV2tEqlPSDVDRmN00WqrP8MJQ1I9qphjHiEQ3kC5E-caxMH9HT4rKdhHzguX2Q",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 49.5221752,
                    "lng": -115.7570894
                },
                "viewport": {
                    "northeast": {
                        "lat": 49.52358242989272,
                        "lng": -115.7556288201073
                    },
                    "southwest": {
                        "lat": 49.52088277010728,


{
    "html_attributions": [],
    "next_page_token": "AeJbb3ec5IhU5bz-dx8WPhTlwRTcoz75CFRvUH1vy23mnfbygAUXCiZB_ZEBliZolPDVbVuHBq_rN8mKcLf7PF_UCVs7jVHjQR9_fOHUwTyvoAWx95c9clk22uaU7EKRaUnh_h2vjk-3BoTOtO_xFBYSFBi8Cv_cVMTZKxlTWKpnvybC1ubH-HVDlXtbQ-Ih2Iy6Wo_-ztnNmdgtzoNchfqt91SA2_si_LRepcUW_JdTvQDhaEVvYVPtonxM7-fmGrd0ZFMXvvh4UXD_9RoRZD48IX6ExClgzeJLFlvvVT5qYvUA_ZSnP_PzkEHtcCq-20756n_70nhxgYCyLyU6LdBGerszRpubSmNZFIDNNK3yXqHTtIIcxG6x71RNAk2RO_l6wayWrTDsCOwWFCnuanZleB8y6-SRYqnWItLKfQ9BXUU2lZm1lIYUtw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 36.9029314,
                    "lng": 50.658175
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.90454612989272,
                        "lng": 50.65965307989272
                    },
                    "southwest": {
                        "lat": 36.90184647010727,
    

{
    "html_attributions": [],
    "next_page_token": "AeJbb3cWxsCRg6hEe7OoACderq_Ads28Rl7o0XgdCv_MZqYSm46C5azlIWin3O9l_Bt-2fyck0HJ_9OTqpzjHe7UC_XUJkT-JZYz9SacbAEe3xIwr5vJHsD7hJH-L9wSunajybGkEAIZQki4kqPQphHbQwCPc3OAT3kEhxrMmz-0JctnKOXxFdayxtI89Pwpl8mmz6e_SplVQjuKYg-TXm-X9NURIOiZxfgpoXVMa6T6daJy0hEeQCM8FEtoF7KMMsP5eB8oAZGqJGKr-08_st_4RiWLzoWiPhFLzkgoZgfeOOLhISTA49yUm_cvPwuN8Dh-bFRw-06fof8ahN-MSieLZPTzpcrU4tPzZWFAvDcfpxDIryFDMlpTGzrTPXs_L5YNmp6-Mh_gdENtJziV4c4FAf88xjIhbbwQ55mW-izx2eAU5GAT5cHqqw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 28.0027579,
                    "lng": -81.72574039999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.00415457989272,
                        "lng": -81.72442112010728
                    },
                    "southwest": {
                        "lat": 28.0014549201

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -8.266608699999999,
                    "lng": -72.74143959999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.265227070107278,
                        "lng": -72.74007697010728
                    },
                    "southwest": {
                        "lat": -8.267926729892721,
                        "lng": -72.74277662989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel Souza",
            "place_id": "ChIJUfAREur9mJERHBlXEWqfs_E",
         

{
    "html_attributions": [],
    "next_page_token": "AeJbb3czdxYFZ2SBZamU2SsEEJzD-RHg41nB0Ka5HaigkzPEb5XmoDzNrQSqAtoMun63e9OqfBC_QY8R6t726beQp7Y1F924NqYsT2a06pgXKifMB9GqgKPqhvoTrUPJVzuY4zHIXo0YAFAw2R7ozh5N4rJoevqQ0RMIvIEEunuspSRysFJb1rkNtP2VjlLh64JEjnTupdpNxlXqlWjl0VdgNNk7XEgSqNtUcD15kSBT_0ygqgoDzWHuH6rzhYaekG1-7sfkGUiFpIhY4Z8hgXUJn1qbHCrHYSAA8fxf4t9xaXIoCwaK07YFCcEODQ2yrLd_s41y9SyQQhfABzyuHnLvyufk_pTamrv7X5iBPFZQEHUnRtJ3MschcVX9-s66VQoXEsQUh59eAZRIdhYjGVJQUjqMF2O_tVb7YT3W_FW5a2ZZ3TSwcVXOqw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.7758937,
                    "lng": -87.11884889999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.77709537989271,
                        "lng": -87.11745222010727
                    },
                    "southwest": {
                        "lat": 37.7743957201

In [13]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country Name,Date,Hotel Name
14,14,burriana,39.8890,-0.0850,77.11,38,0,6.82,ES,1660532111,Sweet Meraki Beach Hotel
28,28,fernie,49.5000,-115.0687,75.40,42,0,3.80,CA,1660532117,Best Western Cranbrook Hotel
322,322,rudsar,37.1376,50.2880,76.98,71,0,3.40,IR,1660532264,Mazandaran Ramsar Parsian Hotel
445,445,auburndale,28.0653,-81.7887,75.88,88,0,3.44,US,1660532055,"Holiday Inn Winter Haven, an IHG Hotel"
446,446,porto walter,-8.2686,-72.7439,73.22,70,0,1.28,BR,1660532325,Hotel Souza
542,542,natal,-5.7950,-35.2094,73.62,83,0,6.91,BR,1660532343,Paua Hotel Boutique
554,554,evansville,37.9748,-87.5558,75.20,86,0,4.61,US,1660531739,"Holiday Inn Owensboro Riverfront, an IHG Hotel"


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country Name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
print(hotel_info)

['\n<dl>\n<dt>Name</dt><dd>Sweet Meraki Beach Hotel</dd>\n<dt>City</dt><dd>burriana</dd>\n<dt>Country</dt><dd>ES</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Best Western Cranbrook Hotel</dd>\n<dt>City</dt><dd>fernie</dd>\n<dt>Country</dt><dd>CA</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Mazandaran Ramsar Parsian Hotel</dd>\n<dt>City</dt><dd>rudsar</dd>\n<dt>Country</dt><dd>IR</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Holiday Inn Winter Haven, an IHG Hotel</dd>\n<dt>City</dt><dd>auburndale</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Hotel Souza</dd>\n<dt>City</dt><dd>porto walter</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Paua Hotel Boutique</dd>\n<dt>City</dt><dd>natal</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Holiday Inn Owensboro Riverfront, an IHG Hotel</dd>\n<dt>City</dt><dd>evansville</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n']


In [19]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Create bank symbol layer
hotels_layer = gmaps.marker_layer(
    locations,
    info_box_content= hotel_info
)


fig = gmaps.figure()
# fig.add_layer(markers)
fig.add_layer(hotels_layer)

fig



Figure(layout=FigureLayout(height='420px'))